In [18]:
import folium as fl
import pandas as pd
import json
import plotly.express as px

In [19]:
crime= pd.read_csv("train.csv")
crime.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [20]:
crimedata2 = pd.DataFrame(crime["PdDistrict"].value_counts().astype(float))

crimedata2 = crimedata2.reset_index()
crimedata2.columns = ["PdDistrict", "Number"]
crimedata2

,PdDistrict,Number
0,SOUTHERN,157182.0
1,MISSION,119908.0
2,NORTHERN,105296.0
3,BAYVIEW,89431.0
4,CENTRAL,85460.0
5,TENDERLOIN,81809.0
6,INGLESIDE,78845.0
7,TARAVAL,65596.0
8,PARK,49313.0
9,RICHMOND,45209.0


In [21]:
grouped= crime.groupby(["PdDistrict"])["X", "Y"].mean()
pd.DataFrame(grouped)

C:\Users\Derrick\AppData\Local\Temp/ipykernel_12172/1913874482.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped= crime.groupby(["PdDistrict"])["X", "Y"].mean()


,X,Y
PdDistrict,,
BAYVIEW,-122.393359,37.742516
CENTRAL,-122.409507,37.798740
INGLESIDE,-122.428734,37.729195
MISSION,-122.419393,37.760396
NORTHERN,-122.426428,37.792330
PARK,-122.445369,37.772418
RICHMOND,-122.469782,37.788294
SOUTHERN,-122.405185,37.782573
TARAVAL,-122.477214,37.740736


In [22]:
df3= grouped.merge(crimedata2, on ="PdDistrict")
df3.rename (columns= {"X": 'x', "Y": "y"}, inplace= True)
df3

,PdDistrict,x,y,Number
0,BAYVIEW,-122.393359,37.742516,89431.0
1,CENTRAL,-122.409507,37.798740,85460.0
2,INGLESIDE,-122.428734,37.729195,78845.0
3,MISSION,-122.419393,37.760396,119908.0
4,NORTHERN,-122.426428,37.792330,105296.0
5,PARK,-122.445369,37.772418,49313.0
6,RICHMOND,-122.469782,37.788294,45209.0
7,SOUTHERN,-122.405185,37.782573,157182.0
8,TARAVAL,-122.477214,37.740736,65596.0
9,TENDERLOIN,-122.412152,37.793377,81809.0


In [23]:
f= open("Districts.geojson")
cc= json.load(f)

sf_map = fl.Map(location=[37.7749,-122.4194],
                    zoom_start = 12.3,
                    )
#bins = [8699, 12648, 16597, 20546, 24495, 28445]

sf_map.choropleth(
    geo_data=cc,
    data = df3,
    columns = ["PdDistrict", "Number"],
    #threshold_scale = bins,
    key_on = 'feature.properties.district',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='San Francisco Crime Incidents'
)

sf_map

H:\ananconda1\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [24]:
#for row in df3.iterrows():
  #  row_values = row[1] 
   # center_point = row_values[]
   # location = [center_point.____, center_point.____]
   # popup = ('Council District: ' + str(row_values[____]) + 
   #          ';  ' + 'permits issued: ' + str(row_values[____]))
   # marker = folium.Marker(location = location, popup = popup)
    #marker.____(m)

In [25]:
f= open("df4.geojson")
cc= json.load(f)

sf_map = fl.Map(location=[37.7749,-122.4194],tiles="OpenStreetMap",
                    zoom_start = 12,
                    )
#bins = [8699, 12648, 16597, 20546, 24495, 28445]

for i in range(0,len(df3)):
    fl.Marker(
      location=[df3.iloc[i]['y'], df3.iloc[i]['x']],
      popup=df3.iloc[i][["Number"]],
      icon=fl.DivIcon(html=f"""<div style="font-family: Fantasy; font-weight:20px;font-size: 20px;color: green">{df3.iloc[i]['Number'], df3.iloc[i]["PdDistrict"]}</div>""")
   ).add_to(sf_map)
sf_map